# RNN Defect Detection per-Window Phase

This notebook extends the defect detection model by computing amplitude and phase **for each sliding window** and incorporating these as additional window-level features into the RNN architecture.

In [15]:
import os
import glob
import pickle
import math
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler

# === Parameters ===
TRAIN_DIR    = 'saved/train_data'
TEST_CSV     = 'saved/test/test.csv'
TEST_DIR    = 'saved/test'
OUTPUT_DIR   = 'saved'
DEVICE       = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

WINDOW_SIZES = [32, 64, 128]
HIDDEN_SIZE  = 64
NUM_LAYERS   = 2
BATCH_SIZE   = 1024
NUM_EPOCHS   = 10
LR           = 1e-3

CHANNEL_IDS  = [0, 1, 2]
MODEL_PATH   = os.path.join(OUTPUT_DIR, 'rnn_defect_model.pth')
SCALER_DIR   = os.path.join(OUTPUT_DIR, 'scalers')
os.makedirs(SCALER_DIR, exist_ok=True)

def get_amp_phase(data, position, width, lookup_size=100):
    """
    Compute amplitude and phase for a window of 'data' centered at 'position' with given 'width'.
    """
    n = len(data)
    p1 = max(0, position - width // 2)
    p2 = min(n - 1, position + width // 2)
    # find extremes on X and Y
    A = B = C = D = p1
    min_x = max_x = data[p1][0]
    min_y = max_y = data[p1][1]
    for i in range(p1+1, p2+1):
        x,y = data[i]
        if x < min_x: min_x, A = x, i
        if x > max_x: max_x, B = x, i
        if y < min_y: min_y, C = y, i
        if y > max_y: max_y, D = y, i
    if C < A: A,C = C,A
    if D < B: B,D = D,B
    if (C - A) > lookup_size:
        tA = (A+C-lookup_size)//2
        tC = (A+C+lookup_size)//2
        A,C = tA,tC
    if (D - B) > lookup_size:
        tB = (B+D-lookup_size)//2
        tD = (B+D+lookup_size)//2
        B,D = tB,tD
    # select farthest points
    max_d = 0
    q1 = q2 = p1
    for s1 in range(A, C+1):
        for s2 in range(B, D+1):
            dx = data[s1][0] - data[s2][0]
            dy = data[s1][1] - data[s2][1]
            d = dx*dx + dy*dy
            if d > max_d:
                max_d, q1, q2 = d, s1, s2
    x1,y1 = data[q1]
    x2,y2 = data[q2]
    amp = math.sqrt((x2-x1)**2 + (y2-y1)**2)
    phase = math.degrees(math.atan2(y2 - y1, x1 - x2)) % 360
    return amp, phase


In [4]:
def fit_scalers(train_dir, channel_ids):
    """Fit StandardScaler on the 6 per-timestep features for each channel."""
    scalers = {}
    for ch in channel_ids:
        cols = [f'{ch}_X', f'{ch}_Y', f'{ch}_dX', f'{ch}_dY', f'{ch}_ddX', f'{ch}_ddY']
        all_feats = []
        for path in glob.glob(os.path.join(train_dir, '*.csv')):
            df = pd.read_csv(path)
            # compute derivatives
            df[f'{ch}_dX']  = df[f'{ch}_X'].diff().fillna(0)
            df[f'{ch}_dY']  = df[f'{ch}_Y'].diff().fillna(0)
            df[f'{ch}_ddX'] = df[f'{ch}_dX'].diff().fillna(0)
            df[f'{ch}_ddY'] = df[f'{ch}_dY'].diff().fillna(0)
            all_feats.append(df[cols].values.astype(np.float32))
        all_feats = np.vstack(all_feats)
        scaler = StandardScaler().fit(all_feats)
        with open(os.path.join(SCALER_DIR, f'scaler_ch{ch}.pkl'), 'wb') as fp:
            pickle.dump(scaler, fp)
        scalers[ch] = scaler
    return scalers


In [5]:
class MultiChannelDefectDataset(Dataset):
    def __init__(self, train_dir, window_sizes, scalers, channel_ids):
        self.window_sizes = window_sizes
        self.scalers = scalers
        self.channel_ids = channel_ids
        self.items = []     # tuples of (feats_scaled, data_xy, labels)
        self.idx_map = []   # (item_idx, row_idx)
        for path in glob.glob(os.path.join(train_dir, '*.csv')):
            df = pd.read_csv(path)
            # compute derivatives
            for ch in channel_ids:
                df[f'{ch}_dX']  = df[f'{ch}_X'].diff().fillna(0)
                df[f'{ch}_dY']  = df[f'{ch}_Y'].diff().fillna(0)
                df[f'{ch}_ddX'] = df[f'{ch}_dX'].diff().fillna(0)
                df[f'{ch}_ddY'] = df[f'{ch}_dY'].diff().fillna(0)
            labels = (df['slice_number'].fillna(0).astype(int) != 0).astype(np.int64).values
            for ch in channel_ids:
                cols = [f'{ch}_X', f'{ch}_Y', f'{ch}_dX', f'{ch}_dY', f'{ch}_ddX', f'{ch}_ddY']
                feats = df[cols].values.astype(np.float32)
                feats = scalers[ch].transform(feats)
                # data_xy needed for phase/amp
                xy = list(zip(df[f'{ch}_X'], df[f'{ch}_Y']))
                self.items.append((feats, xy, labels))
        for item_idx, (_, _, labels) in enumerate(self.items):
            for i in range(len(labels)):
                self.idx_map.append((item_idx, i))
    def __len__(self):
        return len(self.idx_map)
    def __getitem__(self, idx):
        item_idx, row_idx = self.idx_map[idx]
        feats, xy, labels = self.items[item_idx]
        seqs = []
        window_amps = []
        window_phs  = []
        for w in self.window_sizes:
            start = row_idx - w + 1
            if start >= 0:
                window = feats[start:row_idx+1]
                data_win = xy[start:row_idx+1]
            else:
                pad = np.zeros((-start, feats.shape[1]), dtype=np.float32)
                window = np.vstack([pad, feats[0:row_idx+1]])
                data_win = [(0,0)]*(-start) + xy[:row_idx+1]
            amp, ph = get_amp_phase(data_win, len(data_win)-1, w)
            seqs.append(window)
            window_amps.append(amp)
            window_phs.append(ph)
        label = labels[row_idx]
        return seqs, window_amps, window_phs, label


In [6]:
def collate_fn(batch):
    # batch: list of (seqs, amps, phs, label)
    #  - seqs: list of K windows, each window is (w, feat_dim)
    #  - amps: list of K floats
    #  - phs:  list of K floats
    #  - label: scalar

    # 1) stack sequences per window
    seqs_list = list(zip(*[b[0] for b in batch]))  
    #    seqs_list[i] is a tuple of length batch, each element is (w,feat_dim)
    seqs_tensors = [
        torch.tensor(np.stack(win_batch), dtype=torch.float32)
        for win_batch in seqs_list
    ]  # list of K tensors, each (batch, w, feat_dim)

    # 2) build (batch, K) tensors for amps and phs
    amps = torch.tensor([b[1] for b in batch], dtype=torch.float32)  # shape (batch, K)
    phs  = torch.tensor([b[2] for b in batch], dtype=torch.float32)  # shape (batch, K)

    # 3) labels
    labels = torch.tensor([b[3] for b in batch], dtype=torch.float32)  # (batch,)

    return seqs_tensors, amps, phs, labels


In [7]:
class MultiScaleRNNWithPhase(nn.Module):
    def __init__(self, input_size, hidden_size, window_sizes, num_layers):
        super().__init__()
        self.window_sizes = window_sizes
        self.branches = nn.ModuleList([
            nn.LSTM(input_size, hidden_size, num_layers=num_layers,
                    batch_first=True, dropout=0.2)
            for _ in window_sizes
        ])
        # each window: hidden + [amp, phase] => hidden+2
        total_feats = len(window_sizes)*(hidden_size+2)
        self.classifier = nn.Sequential(
            nn.Linear(total_feats, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_size, 1)
        )
    def forward(self, seqs_list, amps, phs):
        hs = []
        for i, lstm in enumerate(self.branches):
            seq = seqs_list[i]
            _, (h_n, _) = lstm(seq)
            h_last = h_n[-1]                 # (batch, hidden_size)
            amp_i = amps[:, i].unsqueeze(1)  # (batch,1)
            ph_i  = phs[:, i].unsqueeze(1)
            hs.append(torch.cat([h_last, amp_i, ph_i], dim=1))
        h = torch.cat(hs, dim=1)
        return self.classifier(h).squeeze(1)


In [6]:
# === Training ===
scalers = fit_scalers(TRAIN_DIR, CHANNEL_IDS)
ds = MultiChannelDefectDataset(TRAIN_DIR, WINDOW_SIZES, scalers, CHANNEL_IDS)
loader = DataLoader(ds, batch_size=BATCH_SIZE, shuffle=True,
                    collate_fn=collate_fn, num_workers=4)

model = MultiScaleRNNWithPhase(input_size=6, hidden_size=HIDDEN_SIZE,
                               window_sizes=WINDOW_SIZES, num_layers=NUM_LAYERS).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
criterion = nn.BCEWithLogitsLoss()

best_loss = float('inf')
for ep in range(1, NUM_EPOCHS+1):
    model.train()
    total_loss = 0.0
    for seqs, amps, phs, labels in tqdm(loader, desc=f'Epoch {ep}'):
        seqs = [s.to(DEVICE) for s in seqs]
        amps = amps.to(DEVICE)
        phs  = phs.to(DEVICE)
        labels = labels.to(DEVICE)
        logits = model(seqs, amps, phs)
        loss = criterion(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 2.0)
        optimizer.step()
        total_loss += loss.item() * labels.size(0)
    avg = total_loss / len(ds)
    print(f'Epoch {ep}, Avg Loss: {avg:.4f}')
    if avg < best_loss:
        best_loss = avg
        torch.save(model.state_dict(), MODEL_PATH)


Epoch 1: 100%|██████████| 594/594 [00:48<00:00, 12.28it/s]


Epoch 1, Avg Loss: 0.1307


Epoch 2: 100%|██████████| 594/594 [00:45<00:00, 13.07it/s]


Epoch 2, Avg Loss: 0.0390


Epoch 3: 100%|██████████| 594/594 [00:45<00:00, 12.98it/s]


...

Epoch 8, Avg Loss: 0.0220


Epoch 9: 100%|██████████| 594/594 [00:45<00:00, 13.10it/s]


Epoch 9, Avg Loss: 0.0201


Epoch 10: 100%|██████████| 594/594 [00:45<00:00, 13.06it/s]


Epoch 10, Avg Loss: 0.0195


In [8]:
# === Inference ===
import pickle

# Load scalers
scalers = {ch: pickle.load(open(os.path.join(SCALER_DIR, f'scaler_ch{ch}.pkl'),'rb'))
           for ch in CHANNEL_IDS}

# Load model
model = MultiScaleRNNWithPhase(input_size=6, hidden_size=HIDDEN_SIZE,
                               window_sizes=WINDOW_SIZES, num_layers=NUM_LAYERS).to(DEVICE)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.eval()

# Read test data
df = pd.read_csv(TEST_CSV)
# compute derivatives
for ch in CHANNEL_IDS:
    df[f'{ch}_dX']  = df[f'{ch}_X'].diff().fillna(0)
    df[f'{ch}_dY']  = df[f'{ch}_Y'].diff().fillna(0)
    df[f'{ch}_ddX'] = df[f'{ch}_dX'].diff().fillna(0)
    df[f'{ch}_ddY'] = df[f'{ch}_dY'].diff().fillna(0)

# prepare per-window data and infer
n = len(df)
for ch in CHANNEL_IDS:
    # get scaled feats and xy
    cols = [f'{ch}_X', f'{ch}_Y', f'{ch}_dX', f'{ch}_dY', f'{ch}_ddX', f'{ch}_ddY']
    feats = df[cols].values.astype(np.float32)
    feats = scalers[ch].transform(feats)
    xy    = list(zip(df[f'{ch}_X'], df[f'{ch}_Y']))
    proba = np.zeros(n, dtype=np.float32)
    with torch.no_grad():
        for start in range(0, n, BATCH_SIZE):
            end = min(n, start + BATCH_SIZE)
            batch_size = end - start
            seqs, amps, phs = [], [], []
            for w in WINDOW_SIZES:
                # build batch window matrix
                mat = np.zeros((batch_size, w, 6), dtype=np.float32)
                batch_amps = []
                batch_phs  = []
                for i in range(batch_size):
                    idx = start + i
                    s = idx - w + 1
                    if s >= 0:
                        window = feats[s:idx+1]
                        data_win = xy[s:idx+1]
                    else:
                        pad = np.zeros((-s,6), dtype=np.float32)
                        window = np.vstack([pad, feats[:idx+1]])
                        data_win = [(0,0)]*(-s) + xy[:idx+1]
                    mat[i] = window
                    amp, ph = get_amp_phase(data_win, len(data_win)-1, w)
                    batch_amps.append(amp)
                    batch_phs.append(ph)
                seqs.append(torch.tensor(mat, dtype=torch.float32).to(DEVICE))
                amps.append(batch_amps)
                phs.append(batch_phs)
            # transpose amps/phs to shape (batch, num_windows)
            amps_t = torch.tensor(np.stack(amps, axis=1), dtype=torch.float32).to(DEVICE)
            phs_t  = torch.tensor(np.stack(phs, axis=1), dtype=torch.float32).to(DEVICE)
            logits = model(seqs, amps_t, phs_t).cpu().numpy()
            proba[start:end] = 1/(1+np.exp(-logits))
    df[f'defect_proba_{ch}'] = proba

# Save results
out_csv = os.path.join(OUTPUT_DIR, os.path.basename(TEST_CSV).replace('.csv','_out.csv'))
df.to_csv(out_csv, index=False)
print(f'Inference results saved to {out_csv}')


Inference results saved to saved/test_out.csv


In [19]:
# === Evaluation Metrics ===
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Prepare your test DataLoader
test_ds     = MultiChannelDefectDataset(TRAIN_DIR, WINDOW_SIZES, scalers, CHANNEL_IDS)
test_loader = DataLoader(
    test_ds,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=collate_fn,
    num_workers=4
)

# Load the best model
model = MultiScaleRNNWithPhase(
    input_size=6,
    hidden_size=HIDDEN_SIZE,
    window_sizes=WINDOW_SIZES,
    num_layers=NUM_LAYERS
).to(DEVICE)
model.load_state_dict(torch.load(MODEL_PATH))
model.eval()

all_labels = []
all_probs  = []

with torch.no_grad():
    for seqs, amps, phs, labels in tqdm(test_loader, desc='Evaluating'):
        # move inputs to device one-by-one for seqs (a list of tensors)
        seqs = [win.to(DEVICE) for win in seqs]
        amps = amps.to(DEVICE)
        phs  = phs.to(DEVICE)

        logits = model(seqs, amps, phs)
        probs  = torch.sigmoid(logits).cpu().numpy().flatten()
        
        all_probs.extend(probs)
        all_labels.extend(labels.numpy().flatten())

# compute metrics
all_labels = np.array(all_labels)
all_preds  = (np.array(all_probs) > 0.5).astype(int)

accuracy  = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds)
recall    = recall_score(all_labels, all_preds)
f1        = f1_score(all_labels, all_preds)
roc_auc   = roc_auc_score(all_labels, all_probs)

print(f'Accuracy : {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall   : {recall:.4f}')
print(f'F1 Score : {f1:.4f}')
print(f'ROC AUC  : {roc_auc:.4f}')


Evaluating: 100%|██████████| 594/594 [00:47<00:00, 12.44it/s]


Accuracy : 0.9927
Precision: 0.9837
Recall   : 0.7119
F1 Score : 0.8260
ROC AUC  : 0.9969
